In [8]:
# standard
import pandas as pd
import numpy as np
import os
import sklearn
import re
# pipelining
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.compose import ColumnTransformer

# constants
data_fname = "../data/all_risk_processed.csv"
os.getcwd()

'/Users/markqiao/Downloads/vm-spinal-risk/vm-spinal-risk/notebooks'

In [2]:
data = pd.read_csv(os.path.join(os.getcwd(), data_fname))
data.shape

(802, 85)

In [17]:
def filter_df_by_attention_check(data:pd.DataFrame, col_start: int, col_end: int, tol:int) -> pd.DataFrame:
    """
    Filter a pandas data frame of survey responses for rows where the attention check is passed.

    args:
        data: pd.DataFrame
            Survey responses. Columns may vary.
        col_start: int
            The column index where the attention check question starts.
        col_end: int 
            The column index where the attention check question ends.
        tol: int
            The exact number of choices that must be selected to pass the attention check

    returns: pd.DataFrame - only survey responses that passed the attention check
    """

    subset = data.iloc[:, col_start:col_end]
    result = (subset == 1).sum(axis=1) == tol
    print(f"{result.mean()*100:.2f} percent of responses passed the attention check.")
    return subset[result].filter(regex="/(attention_check)", axis = 1)

In [10]:
def verify_age(data:pd.DataFrame, col_idx: int) -> pd.DataFrame:
    """
    Filter a pandas data frame of survey responses for rows where the age is reasonable.

    args:
        data: pd.DataFrame
            Survey responses. Columns may vary.
        col_idx: int
            The column index that contain responses for the respondent's age

    returns: pd.DataFrame - only survey responses where the respondent age is reasonable     
    """
    subset = data.iloc[:, col_idx]
    result = (subset.iloc[:, 3] <= 122) | (subset.iloc[:, 3] >= 0)
    print(f"{result.mean()*100:.2f} percent of responses passed the attention check.")
    return result


In [3]:
filter_df_by_attention_check(data, -7, -1, 4).head()

NameError: name 'filter_df_by_attention_check' is not defined

In [4]:
class DropUnbalancedFeatures(BaseEstimator, TransformerMixin):
  """
  Custom scikit-learn transformer to remove features with unbalanced distribution.
  Author: Evan Yip

  Parameters
  ----------
  threshold : float, optional
      The threshold for feature unbalance. Features with a dominant class percentage
      exceeding this threshold will be dropped. Default is 0.9.
  verbose : bool, optional
      If True, print information about the features being removed during fit and
      transformation. Default is True.

  Attributes
  ----------
  threshold : float
      The specified threshold for feature unbalance.
  verbose : bool
      Flag indicating whether to print information during fit and transform.
  columns : list or None
      List of feature names if input data is a DataFrame, otherwise None.
  features_kept : list
      Indices of features kept after applying the unbalance threshold.
  features_dropped : list
      Indices of features dropped after applying the unbalance threshold.

  Methods
  -------
  fit(X, y=None)
      Fit the transformer to the input data, identifying unbalanced features.
  transform(X)
      Transform the input data by keeping only the balanced features.
  get_feature_names_out(input_features=None)
      Return the names of the output features.

  Notes
  -----
  - The transformer identifies features with a dominant class percentage exceeding
    the specified threshold and drops them.
  - The fit method prints information about the removed features if verbose is True.

  Examples
  --------
  >>> transformer = DropUnbalancedFeatures(threshold=0.85, verbose=True)
  >>> X_balanced = transformer.fit_transform(X_unbalanced)
  """
  def __init__(self, threshold=0.9, verbose=True):
    self.threshold = threshold
    self.verbose = verbose
    self.columns = None
    self.features_kept = []
    self.features_dropped = []

  def fit(self, X, y=None):
    if self.verbose:
      print(f"Removing unbalanced features with (threshold={self.threshold})")
    X = X.copy()
    if isinstance(X, pd.DataFrame):
      self.columns = list(X.columns)
    else:
      self.columns = None
    self.features_dropped = []
    for col_idx in range(X.shape[1]):
      counts = np.unique(X.iloc[:, col_idx] if isinstance(X, pd.DataFrame) else X[:, col_idx], return_counts=True)[1]
      percent = counts / counts.sum()

      if (percent > self.threshold).any():
          self.features_dropped.append(col_idx)
      else:
          self.features_kept.append(col_idx)
    if self.verbose:
      print("Parsing complete")
    return self

  def transform(self, X):
    if self.verbose:
      if len(self.features_dropped) > 0:
        print(f"Removed: {self.features_dropped}")
      else:
        print("No features removed")
    return X.iloc[:, self.features_kept] if isinstance(X, pd.DataFrame) else X[:, self.features_kept]

  def get_feature_names_out(self, input_features=None):
    if self.columns is not None:
      output_features = [self.columns[i] for i in self.features_kept]
      return output_features
    else:
      output_features = [input_features[i] for i in self.features_kept]
      return output_features

In [9]:
data.columns

Index(['record_id', 'risk_1_timestamp', 'age', 'sex', 'height', 'weight',
       'zipcode', 'ethnicity', 'income', 'education', 'prior_surg',
       'spin_surg', 'succ_surg', 'religion', 'odi_1', 'odi_2', 'odi_3',
       'odi_4', 'odi_5', 'odi_6', 'odi_7', 'odi_8', 'odi_9', 'odi_10',
       'exer_50improv_1drop', 'exer_50improv_10drop', 'exer_50improv_50drop',
       'exer_50improv_90drop', 'att_check_1', 'exer_90improv_1drop',
       'exer_90improv_10drop', 'exer_90improv_50drop', 'exer_90improv_90drop',
       'exer_50pain_1death', 'exer_50pain_10death', 'exer_50pain_50death',
       'exer_90pain_1death', 'exer_90pain_10death', 'exer_90pain_50death',
       'work_50improv_1drop', 'work_50improv_10drop', 'work_50improv_50drop',
       'work_50improv_90drop', 'work_90improv_1drop', 'work_90improv_10drop',
       'work_90improv_50drop', 'work_50improv_1para', 'work_50improv_10para',
       'work_50improv_50para', 'work_50improv_90para', 'work_90improv_1para',
       'work_90improv_10par

In [15]:
pat_res_risk = data[['exer_50improv_1drop', 'exer_50improv_10drop', 'exer_50improv_50drop',
       'exer_50improv_90drop', 'att_check_1', 'exer_90improv_1drop',
       'exer_90improv_10drop', 'exer_90improv_50drop', 'exer_90improv_90drop',
       'exer_50pain_1death', 'exer_50pain_10death', 'exer_50pain_50death',
       'exer_90pain_1death', 'exer_90pain_10death', 'exer_90pain_50death',
       'work_50improv_1drop', 'work_50improv_10drop', 'work_50improv_50drop',
       'work_50improv_90drop', 'work_90improv_1drop', 'work_90improv_10drop',
       'work_90improv_50drop', 'work_50improv_1para', 'work_50improv_10para',
       'work_50improv_50para', 'work_50improv_90para', 'work_90improv_1para',
       'work_90improv_10para', 'att_check2', 'work_90improv_50para',
       'work_50improv_1death', 'work_50improv_10death',
       'work_50improv_50death', 'work_90improv_1death',
       'work_90improv_10death', 'work_90improv_50death']]
pat_res_risk = pat_res_risk.drop(['att_check_1','att_check2'], axis=1)

In [16]:
pat_res_risk

,exer_50improv_1drop,exer_50improv_10drop,exer_50improv_50drop,exer_50improv_90drop,exer_90improv_1drop,exer_90improv_10drop,exer_90improv_50drop,exer_90improv_90drop,exer_50pain_1death,exer_50pain_10death,...,work_50improv_90para,work_90improv_1para,work_90improv_10para,work_90improv_50para,work_50improv_1death,work_50improv_10death,work_50improv_50death,work_90improv_1death,work_90improv_10death,work_90improv_50death
0,4,4,3,0,5,5,4,4,4,1,...,0,5,5,0,5,3,0,5,5,0
1,4,4,3,2,5,4,3,3,3,2,...,0,5,3,2,4,2,1,5,3,1
2,3,2,0,0,4,3,0,0,0,0,...,0,2,1,0,1,0,0,1,0,0
3,3,2,1,0,5,4,2,0,1,0,...,0,4,3,0,3,0,0,3,2,0
4,5,4,1,0,5,5,3,0,5,4,...,0,5,4,1,5,4,1,5,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
797,2,1,0,0,4,3,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
798,2,1,0,0,4,3,2,1,3,2,...,0,5,3,2,3,2,0,4,3,0
799,1,1,1,0,3,2,1,1,0,0,...,0,1,1,0,1,0,0,1,0,0
800,4,4,0,0,5,5,3,0,4,0,...,0,5,3,0,3,0,0,4,1,0


In [17]:
spinal_risk_list = []
split_names = pat_res_risk.columns[0].split("_")
improv_list = []
comp_list = []
comp_type = []

for col in pat_res_risk.columns:
    split_names = col.split("_")
    match_improv = re.search(r'^(\d+)', split_names[1])
    match_comp = re.search(r'^(\d+)(\w+)', split_names[2])
    improv_list.append(float(match_improv.group(1)))
    comp_list.append(float(match_comp.group(1)))
    if match_comp.group(2) == 'drop':
        comp_type.append(1)
    if match_comp.group(2) == 'para':
        comp_type.append(2.304)
    else:
        comp_type.append(2.534)


In [18]:
for index, row in pat_res_risk.iterrows():
    spinal_risk_sum = 0 
    for i in range(len(pat_res_risk.columns)):
        option = 6-row[i]
        col_risk = ((comp_list[i]/option) + (option/improv_list[i]))*comp_type[i]
        spinal_risk_sum += col_risk
    spinal_risk_list.append(spinal_risk_sum)


In [35]:
data['spinal_risk_final'] = spinal_risk_list

data_final = data.drop(['odi_1', 'odi_2', 'odi_3',
       'odi_4', 'odi_5', 'odi_6', 'odi_7', 'odi_8', 'odi_9', 'odi_10',
       'exer_50improv_1drop', 'exer_50improv_10drop', 'exer_50improv_50drop',
       'exer_50improv_90drop', 'att_check_1', 'exer_90improv_1drop',
       'exer_90improv_10drop', 'exer_90improv_50drop', 'exer_90improv_90drop',
       'exer_50pain_1death', 'exer_50pain_10death', 'exer_50pain_50death',
       'exer_90pain_1death', 'exer_90pain_10death', 'exer_90pain_50death',
       'work_50improv_1drop', 'work_50improv_10drop', 'work_50improv_50drop',
       'work_50improv_90drop', 'work_90improv_1drop', 'work_90improv_10drop',
       'work_90improv_50drop', 'work_50improv_1para', 'work_50improv_10para',
       'work_50improv_50para', 'work_50improv_90para', 'work_90improv_1para',
       'work_90improv_10para', 'att_check2', 'work_90improv_50para',
       'work_50improv_1death', 'work_50improv_10death',
       'work_50improv_50death', 'work_90improv_1death',
       'work_90improv_10death', 'work_90improv_50death', 'att_pass',
       'risk_1_complete','height', 'weight','record_id', 'risk_1_timestamp', 
       'zipcode','age_range', 'postal_code','state_code','city',
       'province', 'province_code','latitude', 'longitude', 'FIPS', 'fips', 'GISJOIN', 'state'], axis=1)

In [66]:
pd.set_option('display.max_columns', None)
data_final['ADI_NATRANK'] = pd.to_numeric(data_final['ADI_NATRANK'], errors='coerce').astype(float).astype('Int64')
data_final['ADI_STATERNK'] = pd.to_numeric(data_final['ADI_STATERNK'], errors='coerce').astype(float).astype('Int64')

In [68]:
data_final

,age,sex,ethnicity,income,education,prior_surg,spin_surg,succ_surg,religion,odi_final,bmi,dospert_ethical,dospert_financial,dospert_health/safety,dospert_recreational,dospert_social,height_m,weight_kg,ADI_NATRANK,ADI_STATERNK,spinal_risk_final
0,25,1,4,4,6,1,0.0,1.0,1,0.5,20.336707,14,21,21,34,37,1.6764,57.152592,34,2,699.446853
1,28,1,4,13,6,1,0.0,1.0,1,1.0,22.032750,17,17,23,25,39,1.8542,75.749864,55,6,791.773156
2,20,2,4,11,6,1,0.0,1.0,10,0.0,19.739486,9,6,7,6,24,1.6256,52.163080,47,3,378.154242
3,31,1,5,6,7,1,0.0,1.0,2,2.5,30.822902,8,10,15,14,25,1.8034,100.243832,1,1,380.923204
4,25,1,2,4,7,1,0.0,1.0,10,0.5,25.164632,11,6,28,17,31,1.8796,88.904032,30,1,469.889836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
797,20,1,5,1,6,1,0.0,1.0,1,0.0,23.220144,9,12,12,11,26,1.8542,79.832192,56,6,352.446333
798,28,2,5,1,6,0,NaN,NaN,2,1.5,27.878432,18,19,26,17,34,1.7018,80.739376,54,7,405.500184
799,44,1,2,4,5,0,NaN,NaN,1,2.5,24.522677,9,6,20,10,23,1.8796,86.636072,94,10,357.732464
800,68,2,1,12,8,1,0.0,1.0,10,1.5,29.131396,6,21,6,11,37,1.7018,84.368112,4,2,427.393573


In [69]:
type(data_final['ADI_NATRANK'].iloc[1])

numpy.int64

In [70]:
ohe_cols = ["religion", "ethnicity"]
cat_cols = ["sex", "income", "education", "prior_surg", "spin_surg", "succ_surg"]
num_cols = ["age", "odi_final", "bmi", "dospert_ethical", "dospert_financial", "dospert_health/safety", "dospert_recreational", "dospert_social", "height_m", "weight_kg", "spinal_risk_final", "ADI_NATRANK", "ADI_STATERNK"]

In [71]:
# define preprocessing pipeline
ohe_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')),
    ('selector', DropUnbalancedFeatures(threshold=0.8, verbose=False))
])

cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('selector', DropUnbalancedFeatures(threshold=0.8, verbose=False))
])

num_pipe = Pipeline([
    ('imputer', IterativeImputer(random_state=52)),
    ('scaler', StandardScaler())
])


preprocessor = ColumnTransformer(
    transformers=[
        ('ohe', ohe_pipe, ohe_cols),
        ('cat', cat_pipe, cat_cols),
        ('num', num_pipe, num_cols)
    ])

In [74]:
preprocessor.fit(data_final)  # Fit the ColumnTransformer to your data
transformed_columns = preprocessor.get_feature_names_out(input_features=data_final.columns)

In [75]:
transformed_columns

array(['ohe__religion_10', 'cat__sex', 'cat__income', 'cat__education',
       'cat__prior_surg', 'cat__succ_surg', 'num__age', 'num__odi_final',
       'num__bmi', 'num__dospert_ethical', 'num__dospert_financial',
       'num__dospert_health/safety', 'num__dospert_recreational',
       'num__dospert_social', 'num__height_m', 'num__weight_kg',
       'num__spinal_risk_final', 'num__ADI_NATRANK', 'num__ADI_STATERNK'],
      dtype=object)

In [72]:
processed_final = preprocessor.fit_transform(data_final)

In [76]:
processed_final_df = pd.DataFrame(processed_final, columns=transformed_columns)

In [77]:
processed_final_df

,ohe__religion_10,cat__sex,cat__income,cat__education,cat__prior_surg,cat__succ_surg,num__age,num__odi_final,num__bmi,num__dospert_ethical,num__dospert_financial,num__dospert_health/safety,num__dospert_recreational,num__dospert_social,num__height_m,num__weight_kg,num__spinal_risk_final,num__ADI_NATRANK,num__ADI_STATERNK
0,0.0,1.0,4.0,6.0,1.0,1.0,-1.452452,-0.842861,-0.996776,0.636641,1.039483,1.070493,2.802070,1.174118,-0.550706,-1.101891,2.368692,-0.449689,-1.116111
1,0.0,1.0,13.0,6.0,1.0,1.0,-1.299856,-0.706328,-0.732841,1.256535,0.379415,1.376294,1.564062,1.477776,1.178413,-0.223616,3.124484,0.322597,0.338531
2,1.0,2.0,11.0,6.0,1.0,1.0,-1.706778,-0.979394,-1.089714,-0.396516,-1.435771,-1.070112,-1.049511,-0.799657,-1.044740,-1.337526,-0.261439,0.028393,-0.752450
3,0.0,1.0,6.0,7.0,1.0,1.0,-1.147260,-0.296729,0.635067,-0.603147,-0.775703,0.153091,0.050940,-0.647828,0.684379,0.933138,-0.238772,-1.663281,-1.479771
4,1.0,1.0,4.0,7.0,1.0,1.0,-1.452452,-0.842861,-0.245463,0.016747,-1.435771,2.140796,0.463610,0.263145,1.425430,0.397604,0.489517,-0.596791,-1.479771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
797,0.0,1.0,1.0,6.0,1.0,1.0,-1.706778,-0.979394,-0.548061,-0.396516,-0.445669,-0.305610,-0.361729,-0.496000,1.178413,-0.030823,-0.471887,0.359373,0.338531
798,0.0,2.0,1.0,6.0,0.0,0.0,-1.299856,-0.569795,0.176854,1.463166,0.709449,1.834995,0.463610,0.718631,-0.303689,0.012019,-0.037583,0.285821,0.702192
799,0.0,1.0,4.0,5.0,0.0,0.0,-0.486011,-0.296729,-0.345363,-0.396516,-1.435771,0.917593,-0.499285,-0.951486,1.425430,0.290497,-0.428614,1.756842,1.793173
800,1.0,2.0,12.0,8.0,1.0,1.0,0.734756,-0.569795,0.371838,-1.016410,1.039483,-1.223012,-0.361729,1.174118,-0.303689,0.183390,0.141639,-1.552954,-1.116111


In [80]:
processed_final_df.to_csv('processed_final.csv', index=False)